In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import requests
from requests import get
import warnings
warnings.filterwarnings("ignore")
from requests import get
from bs4 import BeautifulSoup
import nltk
from nltk.util import *
from nltk.corpus import stopwords
import re
import seaborn as sns
from collections import defaultdict

In [59]:
pages_list=[]                         # Empty list that will be appended with the page count

pages_list.append("https://photo.stackexchange.com/questions?tab=newest&pagesize=50")

for page in list(range(2, 601)):     # Looping over first 400 webpages

  pages_list.append('https://photo.stackexchange.com/questions?tab=newest&page='+str(page))
  
print(len(pages_list))

600


## Web scraping Questions

In [60]:
questions_list=[]                     # Creating an empty list of questions that will be appended with the question counts.

for x in pages_list:                  # Looping over all the pages in the page list
  page = requests.get(x)
  soup = BeautifulSoup(page.text, 'html.parser')

  question_name = soup.find_all('h3')   # Finding the h3 tag that has details of the question
  for question in question_name:
        if question.find('a'):
            questions_list.append(question.find('a').text)

In [61]:
questions_list[:5]

['current community',
 'more stack exchange communities',
 'Is it possible for a camera to wear out if it is used as a webcam all day?',
 'Why does the iPhone SE (2016) have worse image quality compared to the current iPhone 12?',
 'Are any of these photos any good? [closed]']

In [62]:
rmv = ['current community', 'more stack exchange communities']

In [63]:
df = pd.DataFrame(questions_list, columns =['Questions']) 

In [64]:
df = df[~df['Questions'].isin(rmv)]

In [65]:
df.head()

,Questions
2,Is it possible for a camera to wear out if it ...
3,Why does the iPhone SE (2016) have worse image...
4,Are any of these photos any good? [closed]
5,What should I be careful about while buying us...
6,How can I make part of an image transparent in...


In [13]:
df = df[~df['Questions'].isin(rmv)]

In [14]:
df.head()

,Questions
2,Are any of these photos any good? [closed]
3,What should I be careful about while buying us...
4,How can I make part of an image transparent in...
5,How can I create a Lightroom Smart Collection ...
6,Is there a technical reason for modern on-came...


In [66]:
print(len(df))

24268


## Web scraping Tags

In [67]:
def ListToString(string):
  # initialize an empty string 
  string1 = ""
    
  # traverse in the string   
  for element in string:  
      string1 = string1 + element   
     
  return string1

# To get the tags from the webpage
tag=[]

for x in pages_list:
  page = requests.get(x)
  soup = BeautifulSoup(page.text, 'html.parser')

  tags=[]

  for div in soup.find_all('div', {"class": re.compile("^tags")}):      
    tags.append(div.get('class')[1:])

  clean_tags=[]

  for i in tags:
    clean_tags.append(ListToString(i).replace('t-','|'))

  for j in clean_tags:
    tag.append(j[1:])

In [68]:
print(len(tag))
print(tag[:11])

24268
['webcam|durability', 'image-quality|iphone', 'canon|dslr|lighting|focus|macro', 'equipmen|recommendation|fujifilm|used-equipment', 'lightroom|photo-editing', 'lightroom|keywords|smar|collections', 'flash|hotshoe-flash|vintage|sync-speed', 'landscape|prime|focal-distance', 'camera-settings|fujifilm', 'effect|blur', 'nikon|live-view']


In [69]:
print(tag[0])
print(tag[-1])

webcam|durability
autofocus|back-focus|tests|focus-adjust|fron|focus


In [70]:
df['Tags']=tag

In [71]:
df.head()

,Questions,Tags
2,Is it possible for a camera to wear out if it ...,webcam|durability
3,Why does the iPhone SE (2016) have worse image...,image-quality|iphone
4,Are any of these photos any good? [closed],canon|dslr|lighting|focus|macro
5,What should I be careful about while buying us...,equipmen|recommendation|fujifilm|used-equipment
6,How can I make part of an image transparent in...,lightroom|photo-editing


In [72]:
df.tail()

,Questions,Tags
25235,What are alternatives to a tripod when I can't...,equipmen|recommendation|tripod|travel|weight|s...
25236,Should I really lug around a lens cap everywhere?,lens|equipmen|protection
25237,How can I stop my HDR shots looking so fake?,technique|hdr|artifacts|halos
25238,"How can I maximize the “blurry background, sha...",depth-of-field|blur|bokeh|shooting-technique|s...
25239,What methods can be used to micro-adjust autof...,autofocus|back-focus|tests|focus-adjust|fron|f...


In [73]:
len(df)

24268

## Data Preprocessing

In [74]:
df['Questions'] = df['Questions'].str.replace(';','').str.replace(':','').str.replace('/',' or ').str.replace('(','').str.replace(')','').str.replace('[','').str.replace(']','').str.replace('"','').str.replace("'",'').str.replace('.','')

In [75]:
df["Tags"] = [item.replace("|", " __label__") for item in df["Tags"]]
df['Tags'] ='__label__' + df['Tags'].astype(str)
df.head()

,Questions,Tags
2,Is it possible for a camera to wear out if it ...,__label__webcam __label__durability
3,Why does the iPhone SE 2016 have worse image q...,__label__image-quality __label__iphone
4,Are any of these photos any good? closed,__label__canon __label__dslr __label__lighting...
5,What should I be careful about while buying us...,__label__equipmen __label__recommendation __la...
6,How can I make part of an image transparent in...,__label__lightroom __label__photo-editing


In [76]:
columns_titles = ["Tags","Questions"]
df=df.reindex(columns=columns_titles)
df.head()

,Tags,Questions
2,__label__webcam __label__durability,Is it possible for a camera to wear out if it ...
3,__label__image-quality __label__iphone,Why does the iPhone SE 2016 have worse image q...
4,__label__canon __label__dslr __label__lighting...,Are any of these photos any good? closed
5,__label__equipmen __label__recommendation __la...,What should I be careful about while buying us...
6,__label__lightroom __label__photo-editing,How can I make part of an image transparent in...


## Writing a .txt file from our datafram data

In [26]:
f= open("Photo_new.txt","w+") #open the file for writing
f.close() #finally closing the file 

In [27]:
df.to_csv(r'Photo_new.txt', header=None, index=None, sep=' ', mode='a')

In [28]:
with open('Photo_new.txt', 'r') as f, open('Photo_f.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))

In [30]:
!head Photo_f.txt

__label__canon __label__dslr __label__lighting __label__focus __label__macro Are any of these photos any good? closed
__label__equipmen __label__recommendation __label__fujifilm __label__used-equipment What should I be careful about while buying used mirrorless camera and lenses?
__label__lightroom __label__photo-editing How can I make part of an image transparent in Lightroom?
__label__lightroom __label__keywords __label__smar __label__collections How can I create a Lightroom Smart Collection with multiple and or or conditions?
__label__flash __label__hotshoe-flash __label__vintage __label__sync-speed Is there a technical reason for modern on-camera flashes flash duration?
__label__landscape __label__prime __label__focal-distance Fast Prime Wide lens for Landscape Photography and focal distance duplicate
__label__camera-settings __label__fujifilm How to shortcut the “release or focus priority” on the FujiFilm X-T4?
__label__effect __label__blur Blur effect or out of focus?
__label__ni

In [77]:
!wc Photo_f.txt

wc: Photo_f.txt: open: No such file or directory


## Splitting data into Training and validation datasets

In [32]:
train_data= round(len(df)*0.85)
validation_data = round(len(df)*0.15) 

print("Number of records for training dataset are:", train_data)
print("Number of records for validation dataset are:", validation_data)

Number of records for training dataset are: 20626
Number of records for validation dataset are: 3640


In [33]:
training_data =pd.read_table("Photo_f.txt", nrows=train_data,header=None,error_bad_lines=False,sep='\t')

valid_data =pd.read_table("Photo_f.txt",skiprows=train_data,nrows=validation_data,
                          header=None,error_bad_lines=False,sep='\t')

In [34]:
# Saving the training and validation files.

np.savetxt('photo.train', training_data.values, fmt='%s', delimiter='\t')
np.savetxt('photo.val', valid_data.values, fmt='%s', delimiter='\t')

### Training Data

In [35]:
print(training_data.head())
print(len(training_data))

                                                   0
0  __label__canon __label__dslr __label__lighting...
1  __label__equipmen __label__recommendation __la...
2  __label__lightroom __label__photo-editing How ...
3  __label__lightroom __label__keywords __label__...
4  __label__flash __label__hotshoe-flash __label_...
20626


### Validation Data

In [36]:
print(valid_data.head())
print(len(valid_data))

                                                   0
0  __label__artifacts __label__lens-flare __label...
1  __label__color-management __label__white-balan...
2  __label__canon __label__nikon Canon or nikon? ...
3  __label__equipmen __label__recommendation __la...
4  __label__lens __label__canon __label__autofocu...
3640


## Applying FastText

In [37]:
pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [38]:
import fasttext

### Traing 1st model

In [39]:
model_1 = fasttext.train_supervised(input="photo.train")

In [ ]:
model_1.test("photo.val")

## Preprocessing the data

In [41]:
cat Photo_f.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > astronomy_preprocessed.txt

In [42]:
!wc astronomy_preprocessed.txt

   24266  363231 2958939 astronomy_preprocessed.txt


In [43]:
training_data =pd.read_table("photo_preprocessed.txt", nrows=train_data,header=None,error_bad_lines=False,sep='\t')

valid_data =pd.read_table("photo_preprocessed.txt",skiprows=train_data,nrows=validation_data,
                          header=None,error_bad_lines=False,sep='\t')

In [44]:
np.savetxt('photo_preprocessed.train', training_data.values, fmt='%s', delimiter='\t')
np.savetxt('photo_preprocessed.val', valid_data.values, fmt='%s', delimiter='\t')

### Training 2nd model with preprocessed data

In [45]:
model_2 = fasttext.train_supervised(input="photo_preprocessed.train")

In [46]:
model_2.test("photo_preprocessed.val")

(3639, 0.27947238252267104, 0.08788454891116489)

### Training 3rd model with 25 epochs

In [47]:
model_3 = fasttext.train_supervised(input="photo_preprocessed.train", epoch=25)

In [48]:
model_3.test("photo_preprocessed.val")

(3639, 0.5366859027205276, 0.1687694434842724)

### Training 4th model with epoch and learning rate

In [49]:
model_4 = fasttext.train_supervised(input="photo_preprocessed.train", epoch=25, lr=0.3)

In [50]:
model_4.test("photo_preprocessed.val")

(3639, 0.5784556196757351, 0.18190459730383685)

### Training 5th model with epoch, learning rate and n-grams

In [51]:
model_5 = fasttext.train_supervised(input="photo_preprocessed.train", epoch=25, lr=0.3, wordNgrams=2)

In [52]:
model_5.test("photo_preprocessed.val")

(3639, 0.5721352019785655, 0.17991704113377116)

### Training 6th model with epoch, learning rate, loss funtion and n-gram

In [53]:
model_6 = fasttext.train_supervised(input="photo_preprocessed.train", epoch=25, lr=0.3, loss='softmax', wordNgrams=2)

In [54]:
model_6.test("photo_preprocessed.val")

(3639, 0.5743336081341027, 0.1806083650190114)

### Training 7th model with different hyper parameters

In [55]:
model_7 = fasttext.train_supervised(input="photo_preprocessed.train", lr=0.3, epoch=25, bucket=200000, dim=300, loss='softmax', wordNgrams=2)

In [56]:
model_7.test("photo_preprocessed.val")

(3639, 0.5685627919758175, 0.17879363982025578)